# Example: convert DEPHY forcing to WRF-LES forcing

# Code to read COMBLE LES/SCM DEPHY forcing files and write input files for WRF LES (ASCII/NetCDF)
### Contributed by Tim Juliano from NCAR/RAL on 2/2/23

### Import libraries

In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import netCDF4
from netCDF4 import Dataset, date2num,num2date, stringtochar
import os
import datetime as dt

### !!! BEGIN USER MODS !!!
#### How many hours before ice edge do you want to start?
#### Note: t0_h = 0 means you are starting approx. at ice edge, t0_h = 10 means you are starting 10 h upstream (north) of the ice edge
#### Note: t0_h must be an integer

In [18]:
t0_h = 10
if t0_h < 0 or t0_h > 10:
    sys.exit('Error: Please set 0 <= t0_h >= 10')

### !!! END USER MODS !!!

### Set some things

In [19]:
nhrs = 18 + t0_h + 1 # total number of simulation hours, including t0; default is from 2020-03-12 at 14 UTC to 2020-03-14 at 00 UTC
if t0_h == 0:
    start_time = '2020-03-13 00:00:00'
    start_day = 13
    start_hour = 0
else:
    start_time = '2020-03-12 ' + str(24-t0_h) + ':00:00'
    start_day = 12
    start_hour = 24-t0_h
print ('Start time is: ' + start_time)

Start time is: 2020-03-12 14:00:00


### Read forcing file

In [3]:
dephy_filename = 'COMBLE_INTERCOMPARISON_NUDGING_FORCING.nc'
dephy = xr.open_dataset(dephy_filename)
dephy

dephy_sfc_filename = 'COMBLE_INTERCOMPARISON_SFC_FORCING.nc'
dephy_sfc = xr.open_dataset(dephy_sfc_filename)
dephy_sfc

<xarray.Dataset>
Dimensions:  (time: 337)
Coordinates:
  * time     (time) datetime64[ns] 2020-03-12T14:00:00 ... 2020-03-13T18:00:00
Data variables:
    ts       (time) float64 ...
Attributes:
    title:           Time-varying surface conditions for 13 March 2020 COMBLE...
    reference:       https://arm-development.github.io/comble-mip/
    authors:         Timothy W. Juliano (NCAR/RAL, tjuliano@ucar.edu); Floria...
    version:         Created on 2023-02-01
    format_version:  DEPHY SCM format version 1
    script:          create_comble_sfc_forcing_era5ml.ipynb
    startDate:       2020-03-12 14:00:00
    endDate:         2020-03-13 18:00:00
    force_geo:       1
    surfaceType:     ocean
    surfaceForcing:  ts
    lat:             77.6 deg N
    dx:              150 m
    dy:              150 m
    dz:              see zw_grid variable
    nx:              648
    ny:              648
    nz:              160

### Write input sounding file for WRF LES in ASCII format

In [4]:
# use a list to accumulate output ASCII lines
lines_sounding = []

# number of lines to be read
n_z = dephy.dims['lev']

# arrays to be reported in columns
z_arr = dephy.coords['lev'].values
th_arr = dephy['theta'][0,:,0,0].values # K
qt_arr = dephy['qv'][0,:,0,0].values*1000. # g/kg
u_arr = dephy['u'][0,:,0,0].values # m/s
v_arr = dephy['v'][0,:,0,0].values # m/s
ps = dephy['ps'][0,0,0].values # hPa
thetas = dephy['thetas'][0,0,0].values # K

# add a surface point (required by DHARMA code)
vars_arr = np.array([ps,thetas,qt_arr[0]])
str_firstline = np.array2string(vars_arr,formatter={'float_kind':lambda vars_arr:"%11.5f" % vars_arr})[1:-1]
lines_sounding.append(str_firstline)

# do profile
for kk in range(n_z):
    vars_arr = np.array([z_arr[kk],th_arr[kk],qt_arr[kk],u_arr[kk],v_arr[kk]])
    str_vars = np.array2string(vars_arr,formatter={'float_kind':lambda vars_arr:"%11.5f" % vars_arr})[1:-1]
    lines_sounding.append(str_vars)

# write list contents to ASCII file
filename_sounding_LES = 'input_sounding'
with open(filename_sounding_LES,mode='wt',encoding='utf-8') as sounding_file:
    sounding_file.write('\n'.join(lines_sounding))

### Write ozone file for RRTMG
#### Pressure file

In [5]:
# use a list to accumulate output ASCII lines
#lines_pressure = []

# number of lines to be read
#n_z = dephy.dims['pres_o3']

# arrays to be reported in columns
#p_arr = dephy.coords['pres_o3'].values # Pa
#p_arr = p_arr[::-1] / 100. # hPa

# do profile
#for kk in range(n_z):
#    vars_arr = np.array([p_arr[kk]])
#    str_vars = np.array2string(vars_arr,formatter={'float_kind':lambda vars_arr:"%11.4f" % vars_arr})[1:-1]
#    lines_pressure.append(str_vars)

# write list contents to ASCII file
#filename_pressure_LES = 'ozone_plev.formatted'
#with open(filename_pressure_LES,mode='wt',encoding='utf-8') as pressure_file:
#    pressure_file.write('\n'.join(lines_pressure))

#### Ozone file

In [6]:
# use a list to accumulate output ASCII lines
#lines_ozone = []

# number of lines to be read
#n_z = dephy.dims['pres_o3']

#n_months = 12
#n_lat = 64

# arrays to be reported in columns
#o3_arr = dephy['o3'][0,:,0,0].values # kg kg-1
#o3_arr = o3_arr[::-1]

# do profile
#for ii in range(n_months):
#    for jj in range(n_lat):
#        for kk in range(n_z):
#            vars_arr = np.array([o3_arr[kk]])
#            str_vars = np.array2string(vars_arr,formatter={'float_kind':lambda vars_arr:"%.4E" % vars_arr})[1:-1]
#            lines_ozone.append(str_vars)

# write list contents to ASCII file
#filename_ozone_LES = 'ozone.formatted'
#with open(filename_ozone_LES,mode='wt',encoding='utf-8') as ozone_file:
#    ozone_file.write('\n'.join(lines_ozone))

# Write large-scale forcing file for WRF LES in netCDF format

### Pressure gradient forcing or nudging?

In [7]:
ls_forcing = 'nudging'

#### Read forcing winds from forcing file

In [8]:
if ls_forcing == 'geostrophic':
    u_force = dephy['ug'][:,:].values # m/s
    v_force = dephy['vg'][:,:].values # m/s
elif ls_forcing == 'nudging':
    u_force = dephy['u_nudging'][:,:].values # m/s
    v_force = dephy['v_nudging'][:,:].values # m/s
else:
    print ('Not sure what to do!')
    sys.exit()

### Repeat z_ls 1D array to be 2D array for writing

In [9]:
nhrs = dephy.dims['time']
z_ls_write = np.tile(z_arr,(np.shape(u_force)[0],1))[0:nhrs,:]

### Compute wind forcing tendency

In [10]:
delt = 60.*60. # time between profiles (s)

u_tend = np.zeros([np.shape(u_force)[0],np.shape(u_force)[1]])
v_tend = np.zeros([np.shape(u_force)[0],np.shape(u_force)[1]])
z_tend = np.zeros([np.shape(u_force)[0],np.shape(u_force)[1]])

u_tend[0:-1,:] = (u_force[1:,:] - u_force[0:-1,:]) / delt
u_tend[-1,:] = u_tend[-2,:]
v_tend[0:-1,:] = (v_force[1:,:] - v_force[0:-1,:]) / delt
v_tend[-1,:] = v_tend[-2,:]
z_tend[0:-1,:] = (z_ls_write[1:,:] - z_ls_write[0:-1,:]) / delt
z_tend[-1,:] = z_tend[-2,:]

### Set relaxation time scales if nudging

In [11]:
if ls_forcing == 'nudging':
    uv_nudge_asl = 500.
    
    tau_h = 2. # hours
    tau_s = tau_h*60.*60.
    inv_tau = np.zeros([np.shape(u_force)[0],np.shape(u_force)[1]]) # time x height
    inv_tau[:,:] = (1./tau_s)*np.ones(np.shape(u_force)[1])
    for i in np.arange(np.shape(u_force)[0]):
        for j in np.arange(np.shape(u_force)[1]):
            if z_ls_write[i,j] < uv_nudge_asl:
                inv_tau[i,j] = 0.0

 ### Forcing NetCDF name and delete file if already exists

In [12]:
savename = 'input_ls_forcing.nc'

if os.path.exists(savename):
    os.remove(savename)
    print('The file ' + savename + ' has been deleted successfully')

### Create new netcdf file

In [13]:
try: ncfile.close()  # just to be safe, make sure dataset is not already open.
except: pass
ncfile = Dataset('./' + savename,mode='w',format='NETCDF3_CLASSIC') 
#print(ncfile)

### Create dimensions

In [14]:
levs = np.shape(u_force)[1]
lev_dim = ncfile.createDimension('force_layers', levs)      # level axis
time_dim = ncfile.createDimension('Time', None)    # unlimited axis (can be appended to)
datestrlen = ncfile.createDimension('DateStrLen', 19)

### Create global attributes

In [15]:
ncfile.title='AUXILIARY FORCING FOR CRM'

### Create variables

#### Dimensions

In [16]:
time = ncfile.createVariable('Times', 'S1', ('Time','DateStrLen',))

### Add times

In [23]:
#dates = []
#for i in np.arange(nhrs):
#    if nhrs == 19:
#        dates.append(dt.datetime(2020,3,13,i))
#    else:
#        if start_hour+i > 23:
#            dates.append(dt.datetime(2020,3,13,start_hour+i-24))
#        else:
#            dates.append(dt.datetime(2020,3,12,start_hour+i))
#times = date2num(dates, time.units)
#time[:] = times


for i in np.arange(nhrs):
    if nhrs == 19:
        hold = dt.datetime(2020,3,13,i)
    else:
        if start_hour+i > 23:
            hold = dt.datetime(2020,3,13,start_hour+i-24)
        else:
            hold = dt.datetime(2020,3,12,start_hour+i)
    hold2 = hold.strftime('%Y-%m-%d_%H:%M:%S')
    
    time[i,:] = stringtochar(np.array(hold2, 'S19'))

#### Time-varying wind forcing profiles

In [24]:
u_ls = ncfile.createVariable('U_LS', np.float32, ('Time','force_layers',),fill_value=-999.)
u_ls.FieldType = 104
u_ls.MemoryOrder = 'Z  '
u_ls.description = 'large scale zonal wind velocity'
u_ls.units = ''
u_ls.stagger = ''

v_ls = ncfile.createVariable('V_LS', np.float32, ('Time','force_layers',),fill_value=-999.)
v_ls.FieldType = 104
v_ls.MemoryOrder = 'Z  '
v_ls.description = 'large scale meridional wind velocity'
v_ls.units = ''
v_ls.stagger = ''

z_ls = ncfile.createVariable('Z_LS', np.float32, ('Time','force_layers',),fill_value=-999.)
z_ls.FieldType = 104
z_ls.MemoryOrder = 'Z  '
z_ls.description = 'height of forcing time series'
z_ls.units = ''
z_ls.stagger = ''

u_ls_tend = ncfile.createVariable('U_LS_TEND', np.float32, ('Time','force_layers',),fill_value=-999.)
u_ls_tend.FieldType = 104
u_ls_tend.MemoryOrder = 'Z  '
u_ls_tend.description = 'tendency of zonal wind'
u_ls_tend.units = ''
u_ls_tend.stagger = ''

v_ls_tend = ncfile.createVariable('V_LS_TEND', np.float32, ('Time','force_layers',),fill_value=-999.)
v_ls_tend.FieldType = 104
v_ls_tend.MemoryOrder = 'Z  '
v_ls_tend.description = 'tendency of meridional wind'
v_ls_tend.units = ''
v_ls_tend.stagger = ''

z_ls_tend = ncfile.createVariable('Z_LS_TEND', np.float32, ('Time','force_layers',),fill_value=-999.)
z_ls_tend.FieldType = 104
z_ls_tend.MemoryOrder = 'Z  '
z_ls_tend.description = 'height of forcing time series'
z_ls_tend.units = ''
z_ls_tend.stagger = ''

if ls_forcing == 'nudging':
    inv_tau_m = ncfile.createVariable('INV_TAU_M', np.float32, ('Time','force_layers',),fill_value=-999.)
    inv_tau_m.FieldType = 104
    inv_tau_m.MemoryOrder = 'Z  '
    inv_tau_m.description = 'inverse relaxation time for momentum'
    inv_tau_m.units = ''
    inv_tau_m.stagger = ''

### Write data

In [25]:
u_ls[:] = u_force
v_ls[:] = v_force
z_ls[:] = z_ls_write
u_ls_tend[:] = u_tend
v_ls_tend[:] = v_tend
z_ls_tend[:] = z_tend
if ls_forcing == 'nudging':
    inv_tau_m[:] = inv_tau

### Close the file

In [26]:
# first print the Dataset object to see what we've got
print(ncfile)
# close the Dataset.
ncfile.close(); print('Dataset is closed!')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    title: AUXILIARY FORCING FOR CRM
    dimensions(sizes): force_layers(136), Time(29), DateStrLen(19)
    variables(dimensions): |S1 Times(Time,DateStrLen), float32 U_LS(Time,force_layers), float32 V_LS(Time,force_layers), float32 Z_LS(Time,force_layers), float32 U_LS_TEND(Time,force_layers), float32 V_LS_TEND(Time,force_layers), float32 Z_LS_TEND(Time,force_layers), float32 INV_TAU_M(Time,force_layers)
    groups: 
Dataset is closed!


# Write surface forcing file for WRF LES in netCDF format

### Read skin temperature from forcing file

In [27]:
ts_arr = dephy_sfc['ts'][:].values # K
nhrs_sfc = dephy_sfc.dims['time']

### Compute skin temperature tendency

In [28]:
delt = 5.*60. # time between profiles (s)

ts_tend = np.zeros(nhrs_sfc)

ts_tend[0:-1] = (ts_arr[1:] - ts_arr[0:-1]) / delt
ts_tend[-1] = ts_tend[-2]

 ### Forcing NetCDF name and delete file if already exists

In [29]:
savename = 'input_sfc_forcing.nc'

if os.path.exists(savename):
    os.remove(savename)
    print('The file ' + savename + ' has been deleted successfully')

### Create new netcdf file

In [30]:
try: ncfile.close()  # just to be safe, make sure dataset is not already open.
except: pass
ncfile = Dataset('./' + savename,mode='w',format='NETCDF3_CLASSIC') 
#print(ncfile)

### Create dimensions

In [31]:
time_dim = ncfile.createDimension('Time', None)    # unlimited axis (can be appended to)
datestrlen = ncfile.createDimension('DateStrLen', 19)

### Create global attributes

In [32]:
ncfile.title='AUXILIARY FORCING FOR CRM'

### Create variables

#### Dimensions

In [33]:
time = ncfile.createVariable('Times', 'S1', ('Time','DateStrLen',))

### Add times

In [38]:
curr_hh = start_hour
curr_dd = start_day
ii = 0
first_pass = 1
for i in np.arange(nhrs_sfc):
    curr_min = 5*ii
    if curr_min % 60 == 0 and first_pass == 0:
        curr_hh+=1
        ii = 0
        curr_min = 5*ii
        if curr_hh == 24:
            curr_hh = 0
            curr_dd+=1
            
    hold = dt.datetime(2020,3,curr_dd,curr_hh,curr_min)
    hold2 = hold.strftime('%Y-%m-%d_%H:%M:%S')
    
    first_pass = 0
    ii+=1
    
    time[i,:] = stringtochar(np.array(hold2, 'S19'))

#### Time-varying skin temperature

In [39]:
pre_tsk = ncfile.createVariable('PRE_TSK', np.float32, ('Time',),fill_value=-999.)
pre_tsk.FieldType = 104
pre_tsk.MemoryOrder = 'Z  '
pre_tsk.description = 'skin temperature'
pre_tsk.units = ''
pre_tsk.stagger = ''

pre_tsk_tend = ncfile.createVariable('PRE_TSK_TEND', np.float32, ('Time',),fill_value=-999.)
pre_tsk_tend.FieldType = 104
pre_tsk_tend.MemoryOrder = 'Z  '
pre_tsk_tend.description = 'skin temperature'
pre_tsk_tend.units = ''
pre_tsk_tend.stagger = ''

### Write data

In [40]:
pre_tsk[:] = ts_arr
pre_tsk_tend[:] = ts_tend

### Close the file

In [41]:
# first print the Dataset object to see what we've got
print(ncfile)
# close the Dataset.
ncfile.close(); print('Dataset is closed!')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    title: AUXILIARY FORCING FOR CRM
    dimensions(sizes): Time(337), DateStrLen(19)
    variables(dimensions): |S1 Times(Time,DateStrLen), float32 PRE_TSK(Time), float32 PRE_TSK_TEND(Time)
    groups: 
Dataset is closed!


# Write vertical grid file for WRF LES in ASCII format

In [42]:
# create vertical grid file contents as list
lines_zw = []

n_zw = n_z = dephy.dims['zw_grid'] # number of heights

for kk in range(n_zw):
    lines_zw.append(str(dephy['zw_grid'][kk].values))

# write vertical grid to ASCII file
filename_zw_LES = 'input_zw_grid'
with open(filename_zw_LES,mode='wt',encoding='utf-8') as zw_file:
    zw_file.write('\n'.join(lines_zw))